
ИУ5-63Б Кузнецов В.А.

Вариант 9

Для заданного набора данных построить модели регрессии. Для построения моделей используйте методы: **дерево решений и случайный лес**.

Оцените качество моделей на основе подходящих метрик качества (не менее двух метрик). Какие метрики качества Вы использовали и почему?

Какие выводы Вы можете сделать о качестве построенных моделей?

Для построения моделей необходимо выполнить требуемую предобработку данных: заполнение пропусков, кодирование категориальных признаков, и т.д. Для сокращения времени построения моделей можно использовать фрагмент набора данных (например, первые 200-500 строк).

Набор данных **brazilian_houses_to_rent**

city - находится ли недвижимость в городе

area - площадь недвижимости в квадратных метрах

rooms - количество комнат

bathroom - количество ванных комнат

parking spaces - количество парковочных мест

floor - этаж, на котором находится недвижимость

animal - принимает ли недвижимость животных ('acept' - допускается, 'not acept' - не допускается)

furniture - состояние мебели в недвижимости ('furnished' - меблировано, 'not furnished' - не меблировано)

hoa - стоимость услуг по обслуживанию жилого комплекса

rent amount - арендная плата

property tax - налог на недвижимость

fire insurance - стоимость страхования от пожара

total - общая стоимость аренды, включающая все расходы

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import r2_score, mean_absolute_error
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV

RANDOM_STATE = 123

In [2]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


## Подготовка данных

In [3]:
data = pd.read_csv('/content/drive/My Drive/datasets/houses_to_rent.csv', sep=",")

In [4]:
data.shape

(6080, 14)

In [5]:
data.head()

,Unnamed: 0,city,area,rooms,bathroom,parking spaces,floor,animal,furniture,hoa,rent amount,property tax,fire insurance,total
0,0,1,240,3,3,4,-,acept,furnished,R$0,"R$8,000","R$1,000",R$121,"R$9,121"
1,1,0,64,2,1,1,10,acept,not furnished,R$540,R$820,R$122,R$11,"R$1,493"
2,2,1,443,5,5,4,3,acept,furnished,"R$4,172","R$7,000","R$1,417",R$89,"R$12,680"
3,3,1,73,2,2,1,12,acept,not furnished,R$700,"R$1,250",R$150,R$16,"R$2,116"
4,4,1,19,1,1,0,-,not acept,not furnished,R$0,"R$1,200",R$41,R$16,"R$1,257"


In [6]:
data.dtypes

Unnamed: 0         int64
city               int64
area               int64
rooms              int64
bathroom           int64
parking spaces     int64
floor             object
animal            object
furniture         object
hoa               object
rent amount       object
property tax      object
fire insurance    object
total             object
dtype: object

In [7]:
data.isnull().sum()

Unnamed: 0        0
city              0
area              0
rooms             0
bathroom          0
parking spaces    0
floor             0
animal            0
furniture         0
hoa               0
rent amount       0
property tax      0
fire insurance    0
total             0
dtype: int64

In [8]:
data.drop(columns=['Unnamed: 0'], inplace=True)

In [9]:
data['floor'] = data['floor'].replace('-', np.nan).astype(float)
data = data.dropna(axis=0, how='any')

In [10]:
for col in ['hoa', 'rent amount', 'property tax', 'fire insurance', 'total']:
    data[col] = data[col].str.replace('R$', '').str.replace(',', '').replace(['Sem info', 'Incluso'], '0').astype(float)

<ipython-input-10-3a12cd2c7ad0>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[col] = data[col].str.replace('R$', '').str.replace(',', '').replace(['Sem info', 'Incluso'], '0').astype(float)
<ipython-input-10-3a12cd2c7ad0>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[col] = data[col].str.replace('R$', '').str.replace(',', '').replace(['Sem info', 'Incluso'], '0').astype(float)
<ipython-input-10-3a12cd2c7ad0>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a 

In [11]:
categorical_features = ['city', 'animal', 'furniture']
numeric_features = ['area', 'rooms', 'bathroom', 'parking spaces', 'floor', 'hoa', 'rent amount', 'property tax', 'fire insurance']

In [12]:
encoded_df = pd.get_dummies(data[categorical_features], drop_first=True)

In [13]:
scaler = StandardScaler()
scaled = scaler.fit_transform(data[numeric_features])
scaled_df = pd.DataFrame(scaled, columns=numeric_features)

In [14]:
encoded_df.reset_index(drop=True, inplace=True)
scaled_df.reset_index(drop=True, inplace=True)
X = pd.concat([scaled_df, encoded_df], axis=1)

In [15]:
y = data['total']

In [16]:
X.head()

,area,rooms,bathroom,parking spaces,floor,hoa,rent amount,property tax,fire insurance,city,animal_not acept,furniture_not furnished
0,-0.152470,-0.341373,-0.903801,-0.418815,0.394955,-0.194214,-0.989233,-0.052635,-0.946528,0,False,True
1,0.748046,2.580533,2.127851,1.991282,-0.767379,0.603610,0.811706,0.127591,0.817743,1,False,False
2,-0.131086,-0.341373,-0.145888,-0.418815,0.727051,-0.159067,-0.863925,-0.048738,-0.833434,1,False,True
3,-0.273647,-1.315342,-0.903801,-1.222181,-0.933427,-0.312833,-0.587082,-0.063769,-0.562007,1,False,True
4,-0.173854,-1.315342,-0.903801,-0.418815,-0.933427,0.115514,0.228878,-0.017425,0.252271,1,False,False


In [17]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=RANDOM_STATE)

## Обучение

In [18]:
dt = DecisionTreeRegressor(random_state=RANDOM_STATE)
dt.fit(X_train, y_train)
dt_predictions = dt.predict(X_test)

In [19]:
rf = RandomForestRegressor(random_state=RANDOM_STATE)
rf.fit(X_train, y_train)
rf_predictions = rf.predict(X_test)

In [20]:
param_rf = {
    'n_estimators': [50, 100, 200],
}

grid_search_rf = GridSearchCV(estimator=RandomForestRegressor(random_state=RANDOM_STATE),
                              param_grid=param_rf,
                              cv=5, n_jobs=-1)
grid_search_rf.fit(X_train, y_train)

print(f"Лучший параметр: {grid_search_rf.best_params_}")
rf = grid_search_rf.best_estimator_

Лучший параметр: {'n_estimators': 200}


## Оценка

Для оценки моделей были выбраны MAE и $R^{2}$.

MAE - легко интерпретируетмая метрика, которая позволяет понять среднее отклонение в тех же единицах измерения, что и данные (бразильские реалаы).

$R^{2}$ - метрика, по которой можно понять насколько хорошо модель обрабатывает разные данные.

In [21]:
# MAE
print(f"Дерево решений: {mean_absolute_error(y_test, dt_predictions):.4f}")
print(f"Случайный лес: {mean_absolute_error(y_test, rf_predictions):.4f}")

Дерево решений: 218.4331
Случайный лес: 117.8601


In [22]:
# R^2
print(f"Дерево решений: {r2_score(y_test, dt_predictions):.4f}")
print(f"Случайный лес: {r2_score(y_test, rf_predictions):.4f}")

Дерево решений: 0.9894
Случайный лес: 0.9956


## Выводы

Обе метода показали значения $R^{2}$, что указывает на то, что обе модели хорошо объясняют вариации данных.

Случайный лес имеет заметно меньшее значение MAE, то есть его предсказания в среднем ближе к реальным.

Ожидаемо, более сложная модель, случайный лес, показала лучшие результаты. При этом на её обучение ушло больше времени.